# Baseline   

Цель: предсказать для каждого пользователя взятие/ невзятие каждого из четырех продуктов **в течение месяца после отчетной даты**, исторические данные по ним находятся в targets

In [2]:
import numpy as np

import pandas as pd
from pandas.api.types import is_float_dtype, is_integer_dtype

from collections import Counter
from sklearn.utils import resample

from datetime import datetime
from dateutil.relativedelta import relativedelta

import gc
import glob
import pyarrow.parquet as pq
from tqdm import trange, tqdm

C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
from typing import List, Optional, Tuple

In [4]:
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# найтройки
# Убираем ограничение отображемых колонок
pd.set_option("display.max_columns", None)
# Устанавливаем тему по умолчанию
sb_dark = sns.dark_palette('skyblue', 8, reverse=True) # teal
sns.set(palette=sb_dark)

In [6]:
# Включаем tqdm для pandas, чтобы можно было запускать progress_apply() вместо простого apply()
tqdm.pandas() 
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

In [7]:
eps = 1e-6

In [8]:
PATH = ''
PATH_DATASET = PATH + 'datasets/sber_source/'
PATH_DATASET_OUTPUT = PATH + 'datasets/'
PATH_DATASET_EMBEDDINGS = PATH + 'datasets/embeddings/'

PATH_DATASET_TARGET_TRAIN = PATH_DATASET + 'train_target.parquet/'
PATH_DATASET_TARGET_TEST = PATH_DATASET + 'test_target_b.parquet/'

# таргеты
train_target_files = glob.glob(PATH_DATASET_TARGET_TRAIN + '/*.parquet')
test_target_files = glob.glob(PATH_DATASET_TARGET_TEST + '/*.parquet')

len(train_target_files), len(test_target_files)

(11, 11)

In [9]:
# del item_object
# gc.collect()

In [10]:
%%time
# Загружаем факты продаж продуктов по трейн клиентам
train_target_df = pq.read_table(PATH_DATASET_OUTPUT + 'compress_train_target_files_07_06_2024.parquet').to_pandas()
train_target_df = train_target_df.rename(columns={'mon': 'report_next_end'})
train_target_df = train_target_df.reset_index()
train_target_df = train_target_df[['client_id', 'report_next_end', 'target_1', 'target_2', 'target_3', 'target_4']]
train_target_df.shape

CPU times: total: 2.72 s
Wall time: 1.54 s


(10246704, 6)

In [11]:
%%time
# Загружаем факты продаж продуктов по ТЕСТ клиентам
test_target_df = pq.read_table(PATH_DATASET_OUTPUT + 'compress_test_target_files_08_06_2024.parquet').to_pandas()
test_target_df = test_target_df.drop_duplicates(subset=['mon', 'client_id'])
test_target_df = test_target_df.rename(columns={'mon': 'report_next_end'})
test_target_df = test_target_df[['client_id', 'report_next_end', 'target_1', 'target_2', 'target_3', 'target_4']]
test_target_df.shape

CPU times: total: 1.09 s
Wall time: 864 ms


(1407671, 6)

In [12]:
# добавляем предсказательный столбец в тестовые данные, чтобы расчитать таргет-фичи для них. Это последний период + месяц
def add_submit_month(df:pd.DataFrame) -> pd.DataFrame:
    # ищем максимальную дату для каждого client_id
    max_dates = df.groupby('client_id')['report_next_end'].max().reset_index()
    # добавляем один месяц к максимальной дате
    max_dates['new_report_next_end'] = max_dates['report_next_end'] + pd.DateOffset(months=1)
    submit_df = max_dates[['client_id', 'new_report_next_end']].rename(columns={'new_report_next_end': 'report_next_end'})
    df = pd.concat([df, submit_df], ignore_index=True)
    df = df.sort_values(['client_id', 'report_next_end'])
    # возвращаем итоговый датафрейм с добавленой строкой следующего месяца и связку клиент+отчетный период для сабмита
    return df.fillna(0), submit_df

test_target_df, submit_df = add_submit_month(test_target_df)
test_target_df.shape, submit_df.shape

((1548159, 6), (140488, 2))

In [13]:
submit_df['report_next_end'].value_counts()

report_next_end
2023-01-31    48877
2022-11-30    46086
2022-12-30    45525
Name: count, dtype: int64

In [14]:
submit_df['client_id'][:1].item()

'00011c01bb22d8f62d9655f32d123dcca5ae55179f8266bdb8676e25321e8477'

In [15]:
%%time
# Рассчитываем факт приобретения клиентом когда-либо продукта 1 или 2/3/4
def get_group_targets(df:pd.DataFrame) -> pd.DataFrame:
    # Факт приобретения клиентом когда-либо продукта 1 или 2/3/4
    df['is_target'] = df[['target_1', 'target_2', 'target_3', 'target_4']].max(axis=1)
    
    # Расширеный факт приобретения клиентом когда-либо группы продуктов 
    df['is_target_1_2'] = df[['target_1', 'target_2']].max(axis=1)
    df['is_target_1_3'] = df[['target_1', 'target_3']].max(axis=1)
    df['is_target_1_4'] = df[['target_1', 'target_4']].max(axis=1)
    df['is_target_2_3'] = df[['target_2', 'target_3']].max(axis=1)
    df['is_target_2_4'] = df[['target_2', 'target_4']].max(axis=1)
    df['is_target_3_4'] = df[['target_3', 'target_4']].max(axis=1)
    
    df['is_target_123'] = df[['target_1', 'target_2', 'target_3']].max(axis=1)
    df['is_target_134'] = df[['target_1', 'target_3', 'target_4']].max(axis=1)
    df['is_target_124'] = df[['target_1', 'target_2', 'target_4']].max(axis=1)
    df['is_target_234'] = df[['target_2', 'target_3', 'target_4']].max(axis=1)
    
    # Второй расширеный факт приобретения клиентом когда-либо группы продуктов 
    df['is_target_1_and_2'] = np.where(df[['target_1', 'target_2']].sum(axis=1) == 2, 1,0)
    df['is_target_1_and_3'] = np.where(df[['target_1', 'target_3']].sum(axis=1) == 2, 1,0)
    df['is_target_1_and_4'] = np.where(df[['target_1', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_2_and_3'] = np.where(df[['target_2', 'target_3']].sum(axis=1) == 2, 1,0)
    df['is_target_2_and_4'] = np.where(df[['target_2', 'target_4']].sum(axis=1) == 2, 1,0)
    df['is_target_3_and_4'] = np.where(df[['target_3', 'target_4']].sum(axis=1) == 2, 1,0)
    
    # кол-во купленных продуктов
    df['is_target_cnt'] = df[['target_1', 'target_2', 'target_3', 'target_4']].sum(axis=1)

    return df

test_target_df = get_group_targets(test_target_df)
train_target_df = get_group_targets(train_target_df)

train_target_df.shape, test_target_df.shape

CPU times: total: 12.6 s
Wall time: 12.5 s


((10246704, 24), (1548159, 24))

In [16]:
target_columns = ['target_1', 'target_2', 'target_3', 'target_4',
                  'is_target', 'is_target_1_2', 'is_target_1_3',
                  'is_target_1_4', 'is_target_2_3', 'is_target_2_4', 'is_target_3_4',
                  'is_target_1_and_2', 'is_target_1_and_3', 'is_target_1_and_4',
                  'is_target_2_and_3', 'is_target_2_and_4', 'is_target_3_and_4',
                  'is_target_123', 'is_target_134', 'is_target_124', 'is_target_234',
                  'is_target_cnt']
len(target_columns)

22

In [17]:
%%time
# Надо определить для каждого месяца, покупал ли клиент ранее продукт или нет
def is_prebuy_product(df:pd.DataFrame, target_col:str) -> pd.DataFrame:
    # Сортируем по клиенту и отчетному периоду
    df = df.sort_values(['client_id', 'report_next_end'])

    # Создаем новую колонку c фактом предудыщей покупи
    df[f'prebuy_{target_col}'] = (df.groupby('client_id')[target_col]
                                   .cummax()
                                   .shift(1)
                                   .fillna(0)
                                   .astype(int))
    # Кол-во приобретенных продуктов
    df[f'cnt_prebuy_{target_col}'] = (df.groupby('client_id')[target_col]
                                           .cumsum()
                                           .shift(1)
                                           .fillna(0)
                                           .astype(int))    
    return df

for tar_col in tqdm(target_columns):
    test_target_df = is_prebuy_product(df=test_target_df, target_col=tar_col)
    train_target_df = is_prebuy_product(df=train_target_df, target_col=tar_col)
    
train_target_df.shape, test_target_df.shape

100%|██████████| 22/22 [02:57<00:00,  8.07s/it]

CPU times: total: 2min 59s
Wall time: 2min 57s


((10246704, 68), (1548159, 68))

In [18]:
%%time
# Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
def series_to_int(col_df:pd.Series):
    """
    Перевод в целочисленные типы
    """
    min_val = col_df.min()
    max_val = col_df.max()
    if min_val >= -128 and max_val <= 127:
        col_df = col_df.astype('int8')
    elif min_val >= -32768 and max_val <= 32767:
        col_df = col_df.astype('int16')
    elif min_val >= -2147483648 and max_val <= 2147483647:
        col_df = col_df.astype('int32')
    else:
        col_df = col_df.astype('int64')
    return col_df

def compression_df(df:pd.DataFrame(), datetime_cols:List[str]=[], category_cols:List[str]=[]):
    """
    Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
    """
    float64_cols = list(df.select_dtypes(include='float64'))  
    df[float64_cols] = df[float64_cols].astype('float32')
    for col in df.columns:
        if col in category_cols:
            df[col] = df[col].astype('category')
        elif col in datetime_cols:
            if df[col].dtypes == 'object':
                df[col] = pd.to_datetime(df[col])
        # Если колонка содержит числа 
        elif is_integer_dtype(df[col]):
            if df[col].dtypes == 'int8':
                continue
            else:
                df[col] = series_to_int(df[col])
        elif is_float_dtype(df[col]):
            # Возможно ли перевести в число
            if np.array_equal(df[col].fillna(0), df[col].fillna(0).astype(int)):
                df[col] = df[col].fillna(0)
                df[col] = series_to_int(df[col])
    return df
test_target_df = compression_df(test_target_df, 
                            datetime_cols=['report_end' ,'report_next_end'],
                           )
test_target_df.shape

CPU times: total: 578 ms
Wall time: 554 ms


(1548159, 68)

In [19]:
train_target_df = train_target_df.rename(columns={'mon': 'report_next_end'})
train_target_df = train_target_df.set_index(['client_id','report_next_end'])
train_target_df.shape

(10246704, 66)

In [20]:
test_target_df = test_target_df.rename(columns={'mon': 'report_next_end'})
test_target_df = test_target_df.set_index(['client_id','report_next_end'])
test_target_df.shape

(1548159, 66)

In [21]:
submit_df = submit_df.rename(columns={'mon': 'report_next_end'})
submit_df = submit_df.set_index(['client_id','report_next_end'])
submit_df.shape

(140488, 0)

In [22]:
# Формируем тестовый датафрейм на базе сабмит фрейма
submit_test_df = submit_df.merge(test_target_df, left_index=True, right_index=True)
submit_test_df.shape

(140488, 66)

In [23]:
train_target_df.shape

(10246704, 66)

In [26]:
train_target_df.shape

(10246704, 66)

## Загружаем val_clients_df test_clients_df

In [22]:
# def custom_random_undersampling(df:pd.DataFrame, target_col:str, desired_ratio:float):
#     """
#     Кастомный RandomUnderSampling, с указанием доли распредления desired_ratio.
    
#     df: исходный DataFrame.
#     target_col: название столбца с целевой переменной.
#     desired_ratio: желаемое соотношение меньшего класса к большему классу (между 0 и 1).
    
#     return: под-выборка исходного DataFrame с заданным соотношением классов.
#     """
#     # Получаем распределение классов в исходном DataFrame
#     class_counts = Counter(df[target_col])
    
#     # Определяем меньший и больший классы
#     minority_class = min(class_counts, key=class_counts.get)
#     majority_class = max(class_counts, key=class_counts.get)
    
#     # Вычисляем желаемое количество примеров меньшего класса
#     minority_count = class_counts[minority_class]
#     majority_count = int(minority_count / desired_ratio)
    
#     # Создаем под-выборки для меньшего и большего классов
#     minority_subset = df[df[target_col] == minority_class]
#     majority_subset = resample(df[df[target_col] == majority_class],
#                                replace=False,
#                                n_samples=majority_count,
#                                random_state=53)
    
#     # Объединяем под-выборки и сохраняем исходный индекс
#     resampled_df = pd.concat([minority_subset, majority_subset], ignore_index=False)
    
#     return resampled_df

# from sklearn.model_selection import train_test_split
# # Разделение на трейн и тест будет производиться независимо для каждого таргета
# # Разделять будем по клиентам, т.е. важно чтобы в трейне и тесте клиенты не совпадали
# # Поэтому сначала схлопываем клиентов с признаком хотя бы раз покупал продукт в любом месяце
# # Далее делим клиентов стратификацией по факту приобретения продукта
# # Далее возвращаем все отчетные месяцы для клиентов train/test/val выборок
# def split_train_test_val(df:pd.DataFrame, target_column:str='target_1'):
#     client_any_one_target_df = df[[target_column, 'client_id']].groupby('client_id')[target_column].agg(lambda x: any(x)).reset_index()
#     X_train, X_test, _, y_test = train_test_split(client_any_one_target_df['client_id'], 
#                                                         client_any_one_target_df[target_column], 
#                                                         test_size=0.1, 
#                                                         stratify=client_any_one_target_df[target_column], 
#                                                         random_state=53)
#     X_test, X_val, _, _ = train_test_split(X_test, 
#                                            y_test, 
#                                            test_size=0.5, 
#                                            stratify=y_test, 
#                                            random_state=53)

#     X_train = df[df['client_id'].isin(X_train.values)]
#     X_val = df[df['client_id'].isin(X_val.values)]
#     X_test = df[df['client_id'].isin(X_test.values)]

#     assert len(set(X_train['client_id'].values)&set(X_test['client_id'].values)) == 0, 'Ошибка в разделение клиентов X_train и X_test'
#     assert len(set(X_test['client_id'].values)&set(X_val['client_id'].values)) == 0, 'Ошибка в разделение клиентов  X_val и X_test'
#     assert len(set(X_train['client_id'].values)&set(X_val['client_id'].values)) == 0, 'Ошибка в разделение клиентов X_train и X_val'
    
#     return X_train, X_val, X_test

# sampling_train_target_df = custom_random_undersampling(train_target_df, 'is_target', desired_ratio=0.5)
# sampling_train_target_df.shape

# train_label_df, val_label_df, test_label_df = split_train_test_val(
#                             sampling_train_target_df.reset_index()
#                             )
# # train_label_df.shape, val_label_df.shape, test_label_df.shape

# val_clients_df = val_label_df[['client_id', 'report_next_end']].set_index(['client_id', 'report_next_end'])
# test_clients_df = test_label_df[['client_id', 'report_next_end']].set_index(['client_id', 'report_next_end'])
# # val_clients_df.to_csv('val_clients_df.csv')
# # test_clients_df.to_csv('test_clients_df.csv')
# val_clients_df.shape, test_clients_df.shape

In [128]:
# Считываем ранее сохраненные тестовыие и выалидационные наборы для чистоты эскпериментов по поиску лучшего сэмплера
val_clients_df = pd.read_csv('val_clients_df.csv')
test_clients_df = pd.read_csv('test_clients_df.csv')
# val_clients_df = val_clients_df.set_index(['client_id', 'report_next_end'])
# test_clients_df = test_clients_df.set_index(['client_id', 'report_next_end'])
val_clients_df['report_next_end'] = pd.to_datetime(val_clients_df['report_next_end'])
test_clients_df['report_next_end'] = pd.to_datetime(test_clients_df['report_next_end'])
val_clients_df.shape, test_clients_df.shape

((30427, 2), (30191, 2))

In [129]:
# Выбираем только тех клиентов которых нет в зафиксированной валидационной и тестовой выборках
# assert False, "переписать на индекс, чтобы не тянуть все по клиентам вал тест а только то что тестим"
sub_train_target_df = train_target_df.copy()
sub_train_target_df = sub_train_target_df.reset_index()
sub_train_target_df = sub_train_target_df[~(
    (sub_train_target_df['client_id'].isin(val_clients_df['client_id']))|
    (sub_train_target_df['client_id'].isin(test_clients_df['client_id']))
)]

sub_train_target_df.shape

(9728184, 68)

In [130]:
# Определяем клиентов с 1м таргетом
class_1_all_target_df = sub_train_target_df[sub_train_target_df['is_target'] == 1]
# class_1_all_target_df = class_1_all_target_df.set_index(['client_id', 'report_next_end'])
# class_1_all_target_df

In [131]:
# class_1_all_target_df.shape
# class_0_all_target_df.shape

In [132]:
# Добавляем рандомные записи по нулевому классу (двойной размер от тех у которых хотя бы есть один таргет)
random_0_class_df = sub_train_target_df[sub_train_target_df['is_target'] == 0].sample(len(class_1_all_target_df)*2, random_state=53)
random_0_class_df = random_0_class_df[['client_id', 'report_next_end']]
random_0_class_df = random_0_class_df.set_index(['client_id', 'report_next_end'])
# class_0_all_target_df = pd.concat([class_0_all_target_df, random_0_class_df])
# class_0_all_target_df.shape 
random_0_class_df.shape

(364494, 0)

In [133]:
class_1_all_target_df = class_1_all_target_df[['client_id', 'report_next_end']]
class_1_all_target_df = class_1_all_target_df.set_index(['client_id', 'report_next_end'])
class_1_all_target_df.shape

(182247, 0)

In [134]:
# val_clients_df.shape, test_clients_df
val_clients_df = val_clients_df.set_index(['client_id', 'report_next_end'])
test_clients_df = test_clients_df.set_index(['client_id', 'report_next_end'])
val_clients_df.shape, test_clients_df.shape

((30427, 0), (30191, 0))

In [146]:
class_1_all_target_df['type'] = 'any_1_class'
random_0_class_df['type'] = 'random_0_class'
val_clients_df['type'] = 'val'
test_clients_df['type'] = 'test'
submit_df['type'] = 'submit'

In [147]:
# Формируем итоговый датасет вместе с итоговым сэмплем и вместе с тест и вал пользователями 
result_sample_Client_Month_df = pd.concat([class_1_all_target_df, random_0_class_df, val_clients_df, test_clients_df, submit_df])
result_sample_Client_Month_df = result_sample_Client_Month_df.reset_index()
result_sample_Client_Month_df.shape

(747847, 3)

In [148]:
# submit_df.reset_index().info()

In [149]:
# Сохраняем в файл итоговый выбор пар клиент-месяц, с которыми будет работа в дальнейшем (например фичи будут рассчитываться только для них)
result_sample_Client_Month_df.to_parquet(PATH_DATASET_OUTPUT + 'result_sample_Client_Month_df_12_06_2024.parquet')
result_sample_Client_Month_df.shape

(747847, 3)

## Старый сэмплер выбор в 0-ой класс пар таргет=1 и его прошлый месяц где таргет = 0

In [ ]:
assert False

In [40]:
# Определяем клиентов с 1м таргетом
class_1_all_target_df = sub_train_target_df[sub_train_target_df['is_target'] == 1]
# class_1_target_1 = class_1_target_1.reset_index()

# Выбираем пользователей, которые покупали продукт
# И далее выбираем их прошлые месяцы, если они там уже не покупали продукты, то идеальный негативный класс
# т.к. состояние клиентов похоже, но в одном случае не покупалось а через месяц купил, значит что-то изменилось
premon_select = class_1_all_target_df[['client_id', 'report_next_end']]
premon_select['report_next_end'] = premon_select['report_next_end'] - pd.DateOffset(months=1)
prepre_mon_select = class_1_all_target_df[['client_id', 'report_next_end']]
prepre_mon_select['report_next_end'] = prepre_mon_select['report_next_end'] - pd.DateOffset(months=2)

class_0_all_target_df = pd.concat(
                        [premon_select, 
                         prepre_mon_select]).set_index(['client_id', 'report_next_end'])
class_0_all_target_df.shape

(364494, 0)

In [41]:
class_1_all_target_df = class_1_all_target_df.set_index(['client_id', 'report_next_end'])
class_1_all_target_df.shape

(182247, 66)

In [44]:
class_0_all_target_df.shape, class_1_all_target_df.shape

((211174, 66), (182247, 66))

In [43]:
sub_train_target_df = sub_train_target_df.set_index(['client_id', 'report_next_end'])
class_0_all_target_df = sub_train_target_df[
        (sub_train_target_df.index.isin(class_0_all_target_df.index))&
        (sub_train_target_df['is_target'] != 1)
]
class_0_all_target_df.shape

(211174, 66)

In [58]:
# Подмешиваем случайных пользователей, которые и никогда не покупали продукт
# sub_train_target_df
random_0_class_df = sub_train_target_df[sub_train_target_df['is_target'] == 0].sample(30_000)
class_0_all_target_df = pd.concat([class_0_all_target_df, random_0_class_df])
class_0_all_target_df.shape 

(118361, 66)

In [ ]:
class_0_all_target_df.values_

In [65]:
# Возвращаем тест и вал пользователей
# class_1_target_1 = pd.concat([class_1_target_1, class_0_all_target_df])
sub_train_target_df = train_target_df.copy()
sub_train_target_df = sub_train_target_df.reset_index()
sub_train_target_df = sub_train_target_df[(
    (sub_train_target_df['client_id'].isin(val_clients_df['client_id']))|
    (sub_train_target_df['client_id'].isin(test_clients_df['client_id']))
)]
sub_train_target_df = sub_train_target_df.set_index(['client_id', 'report_next_end'])

sub_train_target_df.shape

(518520, 66)

In [67]:
# Формируем итоговый датасет вместе с итоговым сэмплем и вместе с тест и вал пользователями 
result_sample_train_target_df = pd.concat([class_1_all_target_df, class_0_all_target_df, sub_train_target_df])
result_sample_train_target_df.shape

(711725, 66)

In [70]:
train_target_df = result_sample_train_target_df
train_target_df.shape

(711725, 66)

In [ ]:
# Первая версия сэмлера

In [71]:
# def custom_random_undersampling(df:pd.DataFrame, target_col:str, desired_ratio:float):
#     """
#     Кастомный RandomUnderSampling, с указанием доли распредления desired_ratio.
    
#     df: исходный DataFrame.
#     target_col: название столбца с целевой переменной.
#     desired_ratio: желаемое соотношение меньшего класса к большему классу (между 0 и 1).
    
#     return: под-выборка исходного DataFrame с заданным соотношением классов.
#     """
#     # Получаем распределение классов в исходном DataFrame
#     class_counts = Counter(df[target_col])
    
#     # Определяем меньший и больший классы
#     minority_class = min(class_counts, key=class_counts.get)
#     majority_class = max(class_counts, key=class_counts.get)
    
#     # Вычисляем желаемое количество примеров меньшего класса
#     minority_count = class_counts[minority_class]
#     majority_count = int(minority_count / desired_ratio)
    
#     # Создаем под-выборки для меньшего и большего классов
#     minority_subset = df[df[target_col] == minority_class]
#     majority_subset = resample(df[df[target_col] == majority_class],
#                                replace=False,
#                                n_samples=majority_count,
#                                random_state=53)
    
#     # Объединяем под-выборки и сохраняем исходный индекс
#     resampled_df = pd.concat([minority_subset, majority_subset], ignore_index=False)
    
#     return resampled_df

# train_target_df = custom_random_undersampling(train_target_df, 'is_target', desired_ratio=0.5)
# train_target_df.shape